In [1]:
import json
import numpy
from collections import defaultdict
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import time

In [2]:
import os
import baseline
import pandas as pd
import numpy as np
#TEMPORARILY SUPPRESS WARNINGS
import warnings

warnings.filterwarnings("ignore")

# get the actual labels in the validation set

In [3]:
captions_train_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/captions_train2017.json"
captions_val_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/captions_val2017.json"
#captions_train  = json.loads(open(captions_train_path).read())

In [4]:
inst_train_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_train2017.json"
inst_val_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_val2017.json"
#inst_train  = json.loads(open(inst_train_path).read())

In [5]:
def get_electronics_info(captions_path, inst_path):
    captions  = json.loads(open(captions_path).read())
    inst  = json.loads(open(inst_path).read())
    image_df = pd.DataFrame(inst['images'])
    annotation_df = pd.DataFrame(inst['annotations'])
    #the 'id' in image_df needs to be changed to 'image_id' in order to join with annotations_df
    renamed_image_df = image_df.copy(deep=True)
    renamed_image_df.rename(columns={'id':'image_id'}, inplace=True)
    images_and_annotations_df = annotation_df.merge(renamed_image_df,on='image_id', how='left')
    electronics_only_merged_df = images_and_annotations_df.loc[(images_and_annotations_df['category_id'] >= 72) & (images_and_annotations_df['category_id'] <= 77)]
    
    #select all images that contain electronics
    #all_images_with_electronics = list(pd.Series(electronics_only_merged_df['image_id']).unique())
    
    return(electronics_only_merged_df)

In [6]:
train_electronics = get_electronics_info(captions_train_path, inst_train_path)
val_electronics = get_electronics_info(captions_val_path, inst_val_path)

In [7]:
train_electronics.head()

,area,bbox,category_id,id,image_id,iscrowd,segmentation,coco_url,date_captured,file_name,flickr_url,height,license,width
341,9919.64620,"[308.43, 35.25, 126.4, 108.71]",72,28405,275781,0,"[[316.85, 102.67, 308.43, 63.9, 414.61, 35.25,...",http://images.cocodataset.org/train2017/000000...,2013-11-14 23:31:26,000000275781.jpg,http://farm4.staticflickr.com/3170/2601758661_...,375,3,500
342,66958.31445,"[319.77, 9.19, 319.77, 286.26]",72,28473,403107,0,"[[328.39, 10.15, 319.77, 206.42, 616.57, 295.4...",http://images.cocodataset.org/train2017/000000...,2013-11-15 15:01:19,000000403107.jpg,http://farm4.staticflickr.com/3047/3154457418_...,427,3,640
343,15215.62820,"[394.5, 42.74, 143.19, 143.19]",72,28633,203085,0,"[[414.41, 46.53, 514.93, 42.74, 513.04, 93.95,...",http://images.cocodataset.org/train2017/000000...,2013-11-15 13:00:53,000000203085.jpg,http://farm9.staticflickr.com/8065/8201625616_...,422,1,600
344,21650.08250,"[48.9, 27.33, 165.39, 149.57]",72,28660,520862,0,"[[205.66, 156.76, 179.78, 158.2, 191.28, 169.7...",http://images.cocodataset.org/train2017/000000...,2013-11-14 21:33:34,000000520862.jpg,http://farm4.staticflickr.com/3376/3631630894_...,640,2,480
345,3906.61265,"[516.13, 236.31, 63.95, 67.7]",72,28685,225913,0,"[[516.96, 242.13, 516.13, 288.64, 524.02, 291....",http://images.cocodataset.org/train2017/000000...,2013-11-14 22:16:49,000000225913.jpg,http://farm4.staticflickr.com/3152/2642920071_...,480,1,640


In [8]:
val_electronics.head()

,area,bbox,category_id,id,image_id,iscrowd,segmentation,coco_url,date_captured,file_name,flickr_url,height,license,width
48,31935.48505,"[279.11, 209.81, 206.02, 160.44]",72,29131,464476,0,"[[279.11, 370.25, 281.96, 214.56, 283.86, 209....",http://images.cocodataset.org/val2017/00000046...,2013-11-14 18:23:40,000000464476.jpg,http://farm1.staticflickr.com/24/35275654_7d93...,375,3,500
49,2293.67560,"[97.28, 202.37, 60.28, 39.91]",72,29249,68833,0,"[[100.23, 203.21, 133.68, 202.93, 155.88, 202....",http://images.cocodataset.org/val2017/00000006...,2013-11-14 16:32:09,000000068833.jpg,http://farm8.staticflickr.com/7152/6427642465_...,480,1,640
50,265862.40795,"[1.08, 1.72, 574.7, 471.39]",72,29555,433915,0,"[[388.52, 2.8, 567.17, 15.71, 574.71, 234.19, ...",http://images.cocodataset.org/val2017/00000043...,2013-11-14 19:04:41,000000433915.jpg,http://farm4.staticflickr.com/3519/3243850638_...,480,3,640
51,61254.97715,"[3.88, 0.0, 287.98, 251.13]",72,29829,149222,0,"[[4.46, 19.19, 12.67, 95.55, 16.77, 133.31, 33...",http://images.cocodataset.org/val2017/00000014...,2013-11-15 15:11:24,000000149222.jpg,http://farm1.staticflickr.com/23/33195394_22d6...,383,2,500
52,16606.20565,"[145.26, 214.29, 165.39, 107.87]",72,30246,17379,0,"[[284.76, 320.72, 267.51, 322.16, 247.37, 322....",http://images.cocodataset.org/val2017/00000001...,2013-11-15 01:16:56,000000017379.jpg,http://farm6.staticflickr.com/5123/5316824279_...,640,1,478


In [9]:
#extract electronics IDs in each image
eIDs_in_images = val_electronics.groupby('image_id')['category_id'].apply(list).to_dict()

In [10]:
#select all images that contain electronics
train_images_with_electronics = list(pd.Series(train_electronics['image_id']).unique())
val_images_with_electronics = list(pd.Series(val_electronics['image_id']).unique())

In [11]:
val_images_with_electronics[:10]

[464476, 68833, 433915, 149222, 17379, 361586, 127987, 186632, 440475, 368900]

In [12]:
len(val_images_with_electronics)

597

# get the validation images with electronics

In [13]:
val_folder_name = '/mnt/muthderd/MIDS/W210/data/val2017/'
all_val_filenames = os.listdir(val_folder_name)

In [14]:
max_file_id_length = len(all_val_filenames[0]) - 4#subtract 4 for the '.jpg' suffix

In [15]:
files_to_open = []
for image_id in val_images_with_electronics:
    file_id = str(image_id)
    zeros_to_add = max_file_id_length-len(file_id)
    filename = ('0'*zeros_to_add) + file_id + '.jpg'
    files_to_open.append(filename)

In [16]:
files_to_open[:10]

['000000464476.jpg',
 '000000068833.jpg',
 '000000433915.jpg',
 '000000149222.jpg',
 '000000017379.jpg',
 '000000361586.jpg',
 '000000127987.jpg',
 '000000186632.jpg',
 '000000440475.jpg',
 '000000368900.jpg']

# test the baseline

In [17]:
%%capture

results_folder = '/mnt/muthderd/MIDS/W210/data/val_results_2/'
data_folder = '/mnt/muthderd/MIDS/W210/data/val2017/'

#test_filenames = os.listdir(data_folder)
test_filenames = files_to_open#[:25]

all_test_annotations = pd.DataFrame(columns=['image_id', 'class_id', 'scores', 1, 2, 3, 4])

baseline_model = baseline.BaselineClassifier(model='faster_rcnn_resnet101_v1d_coco')

pred_start = time.time()

for image in test_filenames:
    class_ids, scores, bounding_boxes = baseline_model.classify_objects(image, data_folder, results_folder)
    
    class_id_series = pd.Series(class_ids[0].asnumpy().reshape(len(class_ids[0]),))
    score_series = pd.Series(scores[0].asnumpy().reshape(len(scores[0]),))
    b_box_DF = pd.DataFrame(bounding_boxes[0].asnumpy().reshape(len(bounding_boxes[0]),4), columns=[1,2,3,4])
    
    class_id_series[class_id_series>-1]
    score_series[class_id_series>-1]
    b_box_DF[class_id_series>-1]
    
    test_annotations = b_box_DF[class_id_series>-1]
    test_annotations['class_id'] = class_id_series[class_id_series>-1]
    test_annotations['scores'] = score_series[class_id_series>-1]
    
    test_annotations['image_id'] = [image[:-4]]*len(test_annotations)
    test_annotations = test_annotations[['image_id', 'class_id', 'scores', 1, 2, 3, 4]]
    
    all_test_annotations = pd.concat([all_test_annotations, test_annotations], ignore_index=True)
    
print("Total Annotations: ", str(len(all_test_annotations)))

pred_end = time.time()
pred_duration = pred_end - pred_start

In [18]:
pred_duration

6366.461427927017

In [19]:
all_test_annotations[:10]

,image_id,class_id,scores,1,2,3,4
0,000000464476,0.0,0.999175,73.210449,80.432396,159.288040,328.972778
1,000000464476,0.0,0.075649,31.552841,124.691551,55.845779,144.931351
2,000000464476,0.0,0.024890,45.479744,119.495323,60.837555,150.507492
3,000000464476,0.0,0.019374,499.265167,343.942780,569.973877,402.544281
4,000000464476,0.0,0.018579,62.934166,107.181396,92.848885,229.949707
5,000000464476,0.0,0.014831,56.207718,65.805794,90.597382,204.747192
6,000000464476,2.0,0.073346,214.522644,182.376892,286.979828,199.108276
7,000000464476,2.0,0.061099,216.262436,185.535675,250.566238,196.891479
8,000000464476,2.0,0.045528,283.032837,182.610962,302.778320,201.569061
9,000000464476,2.0,0.013150,250.138962,181.044601,283.839142,201.072556


In [20]:
all_test_annotations.to_csv('val_result_annotations_2.csv')

In [21]:
pred_prob_threshold = 0.5
result_annotations = all_test_annotations[all_test_annotations['scores']>pred_prob_threshold]
eIDs_in_results = result_annotations.groupby('image_id')['class_id'].apply(list).to_dict()
#eIDs_in_results

In [22]:
#quick hack to deal with the fact that the class IDs predicted by the baseline model are 10 lower than those in the annotation file
#and to throw out non electronics identified
predicted_eIDs = {}
for image_id in eIDs_in_results.keys():
    corrected_IDs = np.add(eIDs_in_results[image_id],10).astype(int)
    llimit = corrected_IDs >= 72
    ulimit = corrected_IDs <=77
    #print(llimit & ulimit)
    corrected_IDs = corrected_IDs[llimit & ulimit]
    #eIDs_in_results[image_id] = list(corrected_IDs)
    predicted_eIDs[int(image_id)] = list(corrected_IDs)
#predicted_eIDs

In [23]:
total_pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
total_label_counts = [0, 0, 0, 0, 0, 0]
pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
label_counts = [0, 0, 0, 0, 0, 0]
#current counting method will not take into account false positives
for image_id in predicted_eIDs.keys():
    labels = eIDs_in_images[image_id]
    for label in labels:
        total_label_counts[label-72] += 1
    preds = predicted_eIDs[image_id]
    for pred in preds:
        total_pred_counts[pred-72] += 1
    
    unique_labels = list(set(labels))
    unique_preds = list(set(preds))
    for label in unique_labels:
        label_counts[label-72] += 1
    for pred in unique_preds:
        pred_counts[pred-72] += 1

In [24]:
total_pred_counts

[263, 216, 104, 191, 181, 234]

In [25]:
total_label_counts

[288, 231, 106, 283, 153, 262]

In [26]:
np.divide(total_pred_counts, total_label_counts)

array([0.91319444, 0.93506494, 0.98113208, 0.67491166, 1.18300654,
       0.89312977])

In [27]:
pred_counts

[196, 163, 80, 106, 120, 171]

In [28]:
label_counts

[207, 183, 88, 145, 106, 214]

In [29]:
np.divide(pred_counts, label_counts)

array([0.9468599 , 0.89071038, 0.90909091, 0.73103448, 1.13207547,
       0.79906542])

In [30]:
sum(pred_counts)/sum(label_counts)

0.8865323435843054

In [31]:
%%capture

results_folder = '/mnt/muthderd/MIDS/W210/data/val_results_3/'
data_folder = '/mnt/muthderd/MIDS/W210/data/val2017/'

#test_filenames = os.listdir(data_folder)
test_filenames = files_to_open#[:25]

all_test_annotations = pd.DataFrame(columns=['image_id', 'class_id', 'scores', 1, 2, 3, 4])

baseline_model = baseline.BaselineClassifier(model='ssd_512_resnet50_v1_coco')

pred_start = time.time()

for image in test_filenames:
    class_ids, scores, bounding_boxes = baseline_model.classify_objects(image, data_folder, results_folder)
    
    class_id_series = pd.Series(class_ids[0].asnumpy().reshape(len(class_ids[0]),))
    score_series = pd.Series(scores[0].asnumpy().reshape(len(scores[0]),))
    b_box_DF = pd.DataFrame(bounding_boxes[0].asnumpy().reshape(len(bounding_boxes[0]),4), columns=[1,2,3,4])
    
    class_id_series[class_id_series>-1]
    score_series[class_id_series>-1]
    b_box_DF[class_id_series>-1]
    
    test_annotations = b_box_DF[class_id_series>-1]
    test_annotations['class_id'] = class_id_series[class_id_series>-1]
    test_annotations['scores'] = score_series[class_id_series>-1]
    
    test_annotations['image_id'] = [image[:-4]]*len(test_annotations)
    test_annotations = test_annotations[['image_id', 'class_id', 'scores', 1, 2, 3, 4]]
    
    all_test_annotations = pd.concat([all_test_annotations, test_annotations], ignore_index=True)
    
print("Total Annotations: ", str(len(all_test_annotations)))

pred_end = time.time()
pred_duration = pred_end - pred_start

In [32]:
pred_duration

564.4249608516693

In [33]:
all_test_annotations[:10]

,image_id,class_id,scores,1,2,3,4
0,000000464476,62.0,0.994084,377.621948,288.606720,652.204346,508.373383
1,000000464476,0.0,0.991716,69.142776,77.343102,159.968185,331.558929
2,000000464476,62.0,0.315119,367.770813,51.634697,624.718262,280.566498
3,000000464476,6.0,0.197120,195.155273,32.164742,373.132996,200.863678
4,000000464476,0.0,0.105416,527.026184,351.558197,537.178528,362.751556
5,000000464476,0.0,0.098738,520.610962,352.366943,528.946655,362.981628
6,000000464476,0.0,0.096872,530.714355,351.537720,541.468994,362.626953
7,000000464476,0.0,0.090418,548.727539,353.481445,557.438843,363.910278
8,000000464476,62.0,0.084859,195.168060,31.367561,373.370697,204.342865
9,000000464476,62.0,0.081976,363.247223,109.605972,589.127808,501.856873


In [34]:
all_test_annotations.to_csv('val_result_annotations_3.csv')

In [35]:
pred_prob_threshold = 0.5
result_annotations = all_test_annotations[all_test_annotations['scores']>pred_prob_threshold]
eIDs_in_results = result_annotations.groupby('image_id')['class_id'].apply(list).to_dict()
#eIDs_in_results

In [36]:
#quick hack to deal with the fact that the class IDs predicted by the baseline model are 10 lower than those in the annotation file
#and to throw out non electronics identified
predicted_eIDs = {}
for image_id in eIDs_in_results.keys():
    corrected_IDs = np.add(eIDs_in_results[image_id],10).astype(int)
    llimit = corrected_IDs >= 72
    ulimit = corrected_IDs <=77
    #print(llimit & ulimit)
    corrected_IDs = corrected_IDs[llimit & ulimit]
    #eIDs_in_results[image_id] = list(corrected_IDs)
    predicted_eIDs[int(image_id)] = list(corrected_IDs)
#predicted_eIDs

In [37]:
total_pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
total_label_counts = [0, 0, 0, 0, 0, 0]
pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
label_counts = [0, 0, 0, 0, 0, 0]
#current counting method will not take into account false positives
for image_id in predicted_eIDs.keys():
    labels = eIDs_in_images[image_id]
    for label in labels:
        total_label_counts[label-72] += 1
    preds = predicted_eIDs[image_id]
    for pred in preds:
        total_pred_counts[pred-72] += 1
    
    unique_labels = list(set(labels))
    unique_preds = list(set(preds))
    for label in unique_labels:
        label_counts[label-72] += 1
    for pred in unique_preds:
        pred_counts[pred-72] += 1

In [38]:
total_pred_counts

[212, 190, 76, 90, 98, 113]

In [39]:
total_label_counts

[288, 230, 106, 282, 153, 259]

In [40]:
np.divide(total_pred_counts, total_label_counts)

array([0.73611111, 0.82608696, 0.71698113, 0.31914894, 0.64052288,
       0.43629344])

In [41]:
pred_counts

[172, 156, 62, 62, 78, 96]

In [42]:
label_counts

[207, 182, 88, 144, 106, 211]

In [43]:
np.divide(pred_counts, label_counts)

array([0.83091787, 0.85714286, 0.70454545, 0.43055556, 0.73584906,
       0.4549763 ])

In [44]:
sum(pred_counts)/sum(label_counts)

0.6673773987206824